In [1]:
#import packages
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

In [2]:
#import data
df_crime = pd.read_csv('/content/MergedData_2017-2021.csv')
df_covid_ward = pd.read_excel('/content/DC-COVID-19-Data-for-11-5-2021.xlsx'
                              , sheet_name = 'Total Cases by Ward', engine='openpyxl')
df_covid_death = pd.read_excel('/content/DC-COVID-19-Data-for-11-5-2021.xlsx'
                               , sheet_name = 'Lives Lost by Ward', engine='openpyxl')
df_sick_police = pd.read_csv('/content/daily_count_police_out_due_to_covid.csv')

In [3]:
#delete columns not needed from crime data
del df_crime['REPORT_DAT']
del df_crime['BLOCK']
del df_crime['YBLOCK']
del df_crime['XBLOCK']
del df_crime['LATITUDE']
del df_crime['LONGITUDE']
del df_crime['END_DATE']

df_crime = df_crime.rename(columns = {'START_DATE':'date'})

df_crime['date'] = pd.to_datetime(df_crime['date']).dt.date
df_crime['date'] = pd.to_datetime(df_crime['date'])
df_crime.dtypes
df_crime

,SHIFT,METHOD,OFFENSE,WARD,DISTRICT,date
0,MIDNIGHT,OTHERS,THEFT F/AUTO,5.0,5.0,2017-10-28
1,MIDNIGHT,KNIFE,ASSAULT W/DANGEROUS WEAPON,6.0,1.0,2017-10-28
2,MIDNIGHT,OTHERS,THEFT F/AUTO,6.0,1.0,2017-10-28
3,DAY,OTHERS,THEFT F/AUTO,6.0,1.0,2017-10-27
4,DAY,OTHERS,THEFT/OTHER,5.0,5.0,2017-10-28
...,...,...,...,...,...,...
151064,DAY,OTHERS,BURGLARY,5.0,5.0,2021-02-04
151065,DAY,OTHERS,THEFT F/AUTO,7.0,6.0,2021-02-04
151066,DAY,OTHERS,THEFT F/AUTO,5.0,4.0,2021-02-03
151067,DAY,KNIFE,ASSAULT W/DANGEROUS WEAPON,8.0,7.0,2021-02-04


In [4]:
df_crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151069 entries, 0 to 151068
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   SHIFT     151069 non-null  object        
 1   METHOD    151069 non-null  object        
 2   OFFENSE   151069 non-null  object        
 3   WARD      147528 non-null  float64       
 4   DISTRICT  150926 non-null  float64       
 5   date      151064 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 6.9+ MB


In [5]:
df_crime.isna().sum()

SHIFT          0
METHOD         0
OFFENSE        0
WARD        3541
DISTRICT     143
date           5
dtype: int64

In [6]:
#drop records with null records
df_crime = df_crime.dropna(axis=0)
print(df_crime.isna().sum())
print(df_crime.shape)

SHIFT       0
METHOD      0
OFFENSE     0
WARD        0
DISTRICT    0
date        0
dtype: int64
(147385, 6)


In [7]:
#covid case data
df_covid_ward = df_covid_ward.transpose()
df_covid_ward.reset_index(drop = True, inplace= True)
df_covid_ward.columns=('date', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'unknown')
df_covid_ward = df_covid_ward.iloc[1: , :]
df_covid_ward['date'] = pd.to_datetime(df_covid_ward['date'])
df_covid_ward['total_cases'] = (df_covid_ward['ward_1'] + df_covid_ward['ward_2'] + df_covid_ward['ward_3'] + df_covid_ward['ward_4'] + df_covid_ward['ward_5'] + df_covid_ward['ward_6'] +df_covid_ward['ward_7'] + df_covid_ward['ward_8'] +df_covid_ward['unknown'])
df_covid_ward['daily_cases'] = df_covid_ward['total_cases'].diff(1)

del df_covid_ward['ward_1']
del df_covid_ward['ward_2']
del df_covid_ward['ward_3']
del df_covid_ward['ward_4']
del df_covid_ward['ward_5']
del df_covid_ward['ward_6']
del df_covid_ward['ward_7']
del df_covid_ward['ward_8']
del df_covid_ward['unknown']

df_covid_ward.fillna(0, inplace=True)

df_covid_ward

,date,total_cases,daily_cases
1,2020-03-31,586,0
2,2020-04-01,653,67
3,2020-04-02,757,104
4,2020-04-03,902,145
5,2020-04-04,998,96
...,...,...,...
536,2021-11-01,64541,71
537,2021-11-02,64602,61
538,2021-11-03,64708,106
539,2021-11-04,64799,91


In [8]:
#death data
df_covid_death = df_covid_death.transpose()
df_covid_death.reset_index(drop = True, inplace= True)
df_covid_death.columns=('date', 'all', 'ward_1', 'ward_2', 'ward_3', 'ward_4', 'ward_5', 'ward_6'
                      , 'ward_7', 'ward_8', 'Experienced Homelessness', 'unknown', 'na')
df_covid_death = df_covid_death.iloc[1: , :]
df_covid_death['date'] = pd.to_datetime(df_covid_death['date'])
df_covid_death['daily_death'] = df_covid_death['all'].diff(1)

del df_covid_death['ward_1']
del df_covid_death['ward_2']
del df_covid_death['ward_3']
del df_covid_death['ward_4']
del df_covid_death['ward_5']
del df_covid_death['ward_6']
del df_covid_death['ward_7']
del df_covid_death['ward_8']
del df_covid_death['unknown']
del df_covid_death['na']
del df_covid_death['Experienced Homelessness']
del df_covid_death['all']

df_covid_death.fillna(0, inplace=True)

df_covid_death

,date,daily_death
1,2020-04-19,0
2,2020-04-20,7
3,2020-04-21,15
4,2020-04-22,12
5,2020-04-23,14
...,...,...
515,2021-11-01,2
516,2021-11-02,0
517,2021-11-03,0
518,2021-11-04,0


In [9]:
#sick police
df_sick_police = df_sick_police.rename(columns = {'DATE_REPORTED':'date'})
df_sick_police['date'] = pd.to_datetime(df_sick_police['date'])

df_sick_police.fillna(0, inplace=True)

df_sick_police

,date,OUT_POSITIVE_QUARANTINE_PSMPD
0,2020-03-07,0.0
1,2020-03-08,0.0
2,2020-03-09,0.0
3,2020-03-10,0.0
4,2020-03-11,0.0
...,...,...
589,2021-10-17,42.0
590,2021-10-18,43.0
591,2021-10-19,43.0
592,2021-10-20,43.0


In [10]:
#merge the dataframes together
merged = df_crime.merge(df_covid_ward, how='left', on = 'date').merge(df_covid_death, how='left', on = 'date').merge(df_sick_police, how='left', on = 'date')

In [11]:
#getting weekday dummies
merged['weekday'] = merged['date'].dt.dayofweek
dummy0 = pd.get_dummies(merged['weekday'])
dummy0.columns = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']
merged = pd.concat([merged, dummy0] , axis = 1)
del merged['weekday']

#get month dummies
merged['month'] = merged['date'].dt.month
dummy1 = pd.get_dummies(merged['month'])
dummy1.columns = ['JAN', 'FEB','MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP',
                 'OCT', 'NOV', 'DEC']              
merged = pd.concat([merged, dummy1] , axis = 1)
del merged['month']

crime_types = merged['OFFENSE'].unique()
dummy2 = pd.get_dummies(merged['OFFENSE'])
dummy2.columns = crime_types
merged = pd.concat([merged, dummy2] , axis = 1)

merged.columns

Index(['SHIFT', 'METHOD', 'OFFENSE', 'WARD', 'DISTRICT', 'date', 'total_cases',
       'daily_cases', 'daily_death', 'OUT_POSITIVE_QUARANTINE_PSMPD', 'MO',
       'TU', 'WE', 'TH', 'FR', 'SA', 'SU', 'JAN', 'FEB', 'MAR', 'APR', 'MAY',
       'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC', 'THEFT F/AUTO',
       'ASSAULT W/DANGEROUS WEAPON', 'THEFT/OTHER', 'MOTOR VEHICLE THEFT',
       'BURGLARY', 'SEX ABUSE', 'ROBBERY', 'HOMICIDE', 'ARSON'],
      dtype='object')

In [12]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147385 entries, 0 to 147384
Data columns (total 38 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   SHIFT                          147385 non-null  object        
 1   METHOD                         147385 non-null  object        
 2   OFFENSE                        147385 non-null  object        
 3   WARD                           147385 non-null  float64       
 4   DISTRICT                       147385 non-null  float64       
 5   date                           147385 non-null  datetime64[ns]
 6   total_cases                    38189 non-null   float64       
 7   daily_cases                    38189 non-null   float64       
 8   daily_death                    37083 non-null   float64       
 9   OUT_POSITIVE_QUARANTINE_PSMPD  43509 non-null   float64       
 10  MO                             147385 non-null  uint8         
 11  

In [13]:
merged.fillna(0, inplace=True)

In [14]:
count = merged.groupby(['OFFENSE'], as_index=False).agg(
{
    'date':'count',
})
count

,OFFENSE,date
0,ARSON,33
1,ASSAULT W/DANGEROUS WEAPON,7824
2,BURGLARY,6386
3,HOMICIDE,804
4,MOTOR VEHICLE THEFT,12785
5,ROBBERY,9722
6,SEX ABUSE,1061
7,THEFT F/AUTO,46406
8,THEFT/OTHER,62364


In [15]:
merged.loc[merged['daily_cases'] == 0].count()

SHIFT                            109965
METHOD                           109965
OFFENSE                          109965
WARD                             109965
DISTRICT                         109965
date                             109965
total_cases                      109965
daily_cases                      109965
daily_death                      109965
OUT_POSITIVE_QUARANTINE_PSMPD    109965
MO                               109965
TU                               109965
WE                               109965
TH                               109965
FR                               109965
SA                               109965
SU                               109965
JAN                              109965
FEB                              109965
MAR                              109965
APR                              109965
MAY                              109965
JUN                              109965
JUL                              109965
AUG                              109965


In [16]:
merged.isna().sum()

SHIFT                            0
METHOD                           0
OFFENSE                          0
WARD                             0
DISTRICT                         0
date                             0
total_cases                      0
daily_cases                      0
daily_death                      0
OUT_POSITIVE_QUARANTINE_PSMPD    0
MO                               0
TU                               0
WE                               0
TH                               0
FR                               0
SA                               0
SU                               0
JAN                              0
FEB                              0
MAR                              0
APR                              0
MAY                              0
JUN                              0
JUL                              0
AUG                              0
SEP                              0
OCT                              0
NOV                              0
DEC                 

In [17]:
#split into precovid/during covid
#merged.to_csv(r'/Users/samuelclark/Documents/UMBC Data Science /Machine learning/Final Projects/Data/merged.csv')

In [19]:
from google.colab import files
merged.to_csv ('MergedData_covid_police.csv', index = False, header=True)
files.download("MergedData_covid_police.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>